In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5 import uic

from pyqtMedisom import *
import pandas as pd
import os
import numpy as np

import urllib.request
import re
import os
import shutil

In [2]:
#UI파일 연결
#단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
form_class = uic.loadUiType("medison.ui")[0]

In [3]:
path = '/home/ubuntu/mini_project/pill.csv'

if os.path.isfile(path):
    pass
else:
    print("Not found '{}'".format(path))


In [ ]:

#화면을 띄우는데 사용되는 Class 선언
class FindMedison(QMainWindow, form_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.initUI()
        # 약 정보가 들어있는 데이터를 데이터 프레임으로 만들기
        self.df = pd.read_csv(path)
        
        self.result_url = '/home/ubuntu/mini_project/drug_image_dataset/'
        # 이미지를 저장할 경로 지정
    
        if os.path.exists(self.result_url): # 해당 디렉토리가 있다면
            shutil.rmtree(self.result_url) # 디렉토리 지우기

        os.mkdir(self.result_url) # 새로 디렉토리를 만든다

    def initUI(self):
        self.box1.currentTextChanged.connect(self.BackFind)
        self.box2.currentTextChanged.connect(self.BackFind)
        self.box3.currentTextChanged.connect(self.BackFind)
        self.box4.currentTextChanged.connect(self.BackFind)

        self.Fbtn.clicked.connect(self.Findbtnpush)

    def BackFind(self):
        if not(self.box1.currentText() == '전체'):
            a = '제형은 '+self.box1.currentText()+'형 입니다\n'
        if self.box1.currentText() == '전체':
            a = ''

        if not(self.box2.currentText() == '전체'):
            b = '모양은 '+self.box2.currentText()+'이며\n'
        if self.box2.currentText() == '전체':
            b = ''

        if not (self.box3.currentText() == '전체'):
            c = '색깔은 '+ self.box3.currentText() + '색 입니다\n'
            if self.box3.currentText() == '갈색':
                c = '색깔은 ' + self.box3.currentText() + ' 입니다\n'
            if self.box3.currentText() == '남색':
                c = '색깔은 ' + self.box3.currentText() + ' 입니다\n'
            if self.box3.currentText() == '회색':
                c = '색깔은 ' + self.box3.currentText() + ' 입니다\n'
        if self.box3.currentText() == '전체':
            c = ''

        if not (self.box4.currentText() == '전체'):
            d = self.box4.currentText()+'형 무늬가 있어요\n'
        if self.box4.currentText() == '없음':
            d = ''
        if self.box4.currentText() == '전체':
            d = ''

        MainTXT = b+a+c+d
        self.MainTxt.setText('제 약의 '+ MainTXT)


    def Findbtnpush(self):
        dstdf = self.df.loc[self.df['의약품제형'] == self.box2.currentText()]
        dstdf = dstdf.loc[dstdf['색상앞'] == self.box3.currentText()]
        dstdf = dstdf.loc[dstdf['제형코드명'] == self.box1.currentText()]
        dstdf = dstdf.loc[dstdf['표기내용앞'] == self.box4.currentText()]
        
        self.ImgSave(dstdf)
        
    def ImgSave(self,dstdf):
        #len(df['큰제품이미지']) : 23257    
        for i in range(len(dstdf['큰제품이미지'])):
            filename = os.path.join(self.result_url,re.split('[<,>,\[,\],/,-,(,). :]',dstdf['품목명'][i])[0]+'.jpg')
            # ~urlretrieve(a,b) : a에 가서 이미지를 b로 저장 
            urllib.request.urlretrieve(dstdf['큰제품이미지'][i], filename)
            #re.split('[/,-,(,). :]' : 품목명에 아스코푸정(히벤즈산티페피딘) --> 아스코푸정 이렇게 나오도록 하려고 해준다

In [ ]:
if __name__ == "__main__" :
    # 약 정보가 들어있는 데이터를 데이터 프레임으로 만들기
    df = pd.read_csv(path)

    app = QApplication(sys.argv)
    myMedison = FindMedison()
    myMedison.show()
    app.exec_()


In [4]:
df = pd.read_csv(path)

In [5]:
df.tail(2)

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,Unnamed: 29
23255,202004930,뉴락스에프정,19580005,태극제약(주),녹색의원형장용성필름코팅정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,TGNF,-,원형,초록,...,20200707,필름코팅정,-,-,-,-,-,-,-,NaN
23256,202003800,케이셋세미정,20191087,주식회사케이에스제약,연한황색의장방형필름코팅정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크,5,장방형,주황,...,20200526,필름코팅정,-,-,-,-,-,-,-,NaN


In [6]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23257 entries, 0 to 23256
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   품목일련번호          23257 non-null  int64  
 1   품목명             23257 non-null  object 
 2   업소일련번호          23257 non-null  int64  
 3   업소명             23257 non-null  object 
 4   성상              23257 non-null  object 
 5   큰제품이미지          23257 non-null  object 
 6   표시앞             23256 non-null  object 
 7   표시뒤             23255 non-null  object 
 8   의약품제형           23257 non-null  object 
 9   색상앞             23257 non-null  object 
 10  색상뒤             23257 non-null  object 
 11  분할선앞            23257 non-null  object 
 12  분할선뒤            23257 non-null  object 
 13  크기장축            23257 non-null  object 
 14  크기단축            23257 non-null  object 
 15  크기두께            23257 non-null  object 
 16  이미지생성일자(약학정보원)  23257 non-null  int64  
 17  분류번호            23257 non-null 

In [8]:
dfImg = pd.DataFrame(df[['품목명','큰제품이미지']])
dfImg.head()

,품목명,큰제품이미지
0,징카민정40mg(은행엽건조엑스)(수출명:징카프란정),https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...
1,아스코푸정(히벤즈산티페피딘),https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...
2,진셀몬정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...
3,저니스타서방정8밀리그램(히드로모르폰염산염),https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...
4,씨즈날정(세티리진염산염),https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...


In [9]:
dfImg['큰제품이미지'][0]

'https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/1Muwq7fAuBq'

In [47]:
df.loc[df['색상앞'] == '하양'].index

Int64Index([    4,     5,     6,     8,     9,    14,    15,    16,    18,
               24,
            ...
            23226, 23233, 23237, 23238, 23243, 23247, 23250, 23251, 23252,
            23253],
           dtype='int64', length=9054)

# 이미지 저장

In [ ]:
import urllib.request
import re
import os
import shutil

result_url = '/home/ubuntu/mini_project/drug_image_dataset/'# 이미지를 저장할 경로 지정
    
if os.path.exists(result_url): # 해당 디렉토리가 있다면
    shutil.rmtree(result_url) # 디렉토리 지우기

os.mkdir(result_url) # 새로 디렉토리를 만든다
    
#len(df['큰제품이미지']) : 23257    
for i in range(len(dfImg['큰제품이미지'])):
    
    filename = os.path.join(result_url,re.split('[<,>,\[,\],/,-,(,). :]',dfImg['품목명'][i])[0]+'.jpg')
    # ~urlretrieve(a,b) : a에 가서 이미지를 b로 저장 
    urllib.request.urlretrieve(dfImg['큰제품이미지'][i], filename)
    #re.split('[/,-,(,). :]' : 품목명에 아스코푸정(히벤즈산티페피딘) --> 아스코푸정 이렇게 나오도록 하려고 해준다

In [28]:
result_url = '/home/ubuntu/mini_project/drug_image_dataset/'
print('약 이미지의 갯수 :', len(os.listdir(result_url)))

약 이미지의 갯수 : 707


In [43]:
result_url


'/home/ubuntu/mini_project/drug_image_dataset/'

In [36]:
os.listdir(result_url)[0]

'파리아톤정10mg.jpg'

In [44]:
# 의약품제형별 갯수
df['의약품제형'].value_counts()

원형      9187
장방형     7430
타원형     5324
기타       484
삼각형      238
팔각형      224
사각형      214
오각형       76
육각형       51
마름모형      27
반원형        2
Name: 의약품제형, dtype: int64

In [ ]:
# 컬러별 갯수
df['색상앞'].value_counts()

In [ ]:
# 약의 분류명 갯수
df['분류명'].value_counts()

In [ ]:
# 표기내용앞 별 갯수
df['표기내용앞'].value_counts()

In [ ]:
df['표기내용앞'].value_counts()

In [17]:
a = df['제형코드명'].value_counts()
a.head()

필름코팅정        11081
나정            4000
경질캡슐제|산제      2838
연질캡슐제|현탁상      960
연질캡슐제|액상       906
Name: 제형코드명, dtype: int64

In [23]:
dstdf = df.loc[df['색상앞'] == '하양']
dstdf.head()

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,Unnamed: 29
4,199703153,씨즈날정(세티리진염산염),19910005,(주)녹십자,흰색의타원형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,CZT,분할선,장방형,하양,...,19971118,필름코팅정,-,-,-,-,-,-,20171113,NaN
5,200004062,라니타드정,19910005,(주)녹십자,백색의원형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,RNT,-,원형,하양,...,20000103,필름코팅정,-,-,-,-,-,-,-,NaN
6,200402488,알레기살정10밀리그람(페미로라스트칼륨),19660001,현대약품(주),띠모양할선이있는황백색원형정제이다,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크226,분할선,원형,하양,...,20040325,나정,↔|+,-,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,-,r0128,-,-,NaN
8,199702297,로부펜정(록소프로펜나트륨수화물),19600001,삼아제약(주),흰색의원형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크,LT,원형,하양,...,19970221,나정,SA|5A|SΛ|VS|5Λ,-,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,-,r0501,-,20190319,NaN
9,200502773,메트그린에스알정500밀리그람(염산메트포르민),19910005,(주)녹십자,백색의양쪽이볼록한장방형의필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,MSR,-,타원형,하양,...,20051111,서방성필름코팅정,-,-,-,-,-,-,20180809,NaN


In [24]:
dstdf=dstdf.loc[dstdf['의약품제형'] == '장방형']
dstdf=dstdf.loc[dstdf['제형코드명'] == '필름코팅정']
#dstdf=dstdf.loc[dstdf['표기내용앞'] == '전체']
dstdf.head(2)

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,Unnamed: 29
4,199703153,씨즈날정(세티리진염산염),19910005,(주)녹십자,흰색의타원형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,CZT,분할선,장방형,하양,...,19971118,필름코팅정,-,-,-,-,-,-,20171113,NaN
65,200700844,씨엠지모사프리드정(모사프리드시트르산염수화물),19640004,(주)씨엠지제약,흰색의장방형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,CP7,분할선,장방형,하양,...,20070103,필름코팅정,-,-,-,-,-,-,20150501,NaN


In [25]:
dstdf.reset_index(drop=True,inplace=True)
dstdf.head()

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,Unnamed: 29
0,199703153,씨즈날정(세티리진염산염),19910005,(주)녹십자,흰색의타원형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,CZT,분할선,장방형,하양,...,19971118,필름코팅정,-,-,-,-,-,-,20171113,NaN
1,200700844,씨엠지모사프리드정(모사프리드시트르산염수화물),19640004,(주)씨엠지제약,흰색의장방형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,CP7,분할선,장방형,하양,...,20070103,필름코팅정,-,-,-,-,-,-,20150501,NaN
2,199000363,싸이신정500밀리그램(시프로플록사신염산염),19540006,일동제약(주),백색의타원형필름제피정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,IDCC,-,장방형,하양,...,19900512,필름코팅정,-,-,-,-,-,-,20200521,NaN
3,200401321,속코정,19570013,일양약품(주),흰색의장방형필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,SOCO,I분할선Y,장방형,하양,...,20040826,필름코팅정,-,-,-,-,-,-,20191104,NaN
4,199700120,세트린정(세티리진염산염)(수출명:ANTIRIZINtab),19860001,한국휴텍스제약(주),흰색의장방형필름코팅정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,HT,CR,장방형,하양,...,19970711,필름코팅정,-,-,-,-,-,-,20170901,NaN


In [48]:
for i in range(len(dstdf)):
    medisonName = re.split('[<,>,\[,\],/,-,(,). :]', dstdf['품목명'][i])[0] + '.jpg\n'
    print(medisonName)

씨즈날정.jpg

씨엠지모사프리드정.jpg

싸이신정500밀리그램.jpg

속코정.jpg

세트린정.jpg

가스프리드정.jpg

졸피람정10밀리그램.jpg

레나젤정400.jpg

부루펜정200밀리그램.jpg

부루펜정400밀리그램.jpg

부루펜정600밀리그람.jpg

본비바정150밀리그람.jpg

졸로푸트정50밀리그람.jpg

한풍안토시드정.jpg

세프로심정.jpg

그래칼정.jpg

가스프린정.jpg

모사로정.jpg

칼테오-80정.jpg

지르텍정.jpg

뉴트로필정.jpg

아모크라정625밀리그램.jpg

동광염산세티리진정.jpg

트리부틴서방정.jpg

토실린정.jpg

에스텔정.jpg

벤다라인정250밀리그램.jpg

세티신정.jpg

글루파콤비정500.jpg

벤다라인정.jpg

부광미졸렌정10밀리그램.jpg

소마지나정500밀리그램.jpg

후라젠틸정.jpg

테메졸정.jpg

코애드에프정.jpg

보령에바스텔정.jpg

보람페플록사신정400mg.jpg

마토코정.jpg

이트코나정.jpg

포나민정.jpg

펜타우드정.jpg

오라빌정.jpg

카멘정5밀리그람.jpg

리렉사정.jpg

엘프로정.jpg

라니쿨정.jpg

쎄르텍정.jpg

알러잘정.jpg

팜시론정.jpg

신일세티리진정.jpg

미토카정.jpg

글루파정850mg.jpg

시노피드플러스정.jpg

글루파정1|000㎎.jpg

하디코정.jpg

모사르정.jpg

파소질정300밀리그람.jpg

아르니펜정.jpg

팜비클로정750밀리그람.jpg

에이드정.jpg

모사피아정.jpg

글루파정500mg.jpg

카리진정.jpg

니칼정.jpg

라니트린정.jpg

클라멘틴정.jpg

코프엔드정.jpg

토루판정.jpg

아목시클정.jpg

조아이부프로펜400밀리그람정.jpg

콤비-코정.jpg

클라목실정.jpg

케이모틴정.jpg

팜시클정750밀리그람.jpg

알러텍정.jpg

유리스코정400밀리그람.jpg

나프록스정275mg.jpg

로텍정.jpg

올코정.jpg

지

In [61]:
dstdf['품목명'][]


'씨즈날정(세티리진염산염)'

In [70]:
result_url = '/home/ubuntu/mini_project/'
i in os.listdir(result_url)
tt = os.path.join(result_url,i)
print('join',tt)
print('11',i)


join /home/ubuntu/mini_project/.idea
11 .idea
